In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [ ]:
import pandas as pd

from lattereview.providers import OpenAIProvider
from lattereview.providers import OllamaProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer

## Testing the base functionalities

Testing the OpenAI provider (with OpenAI and Gemini models):

In [4]:
openanai_provider = OpenAIProvider(model="gpt-4o-mini")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question))

('The capital of France is Paris.',
 {'input_cost': 1.05e-06, 'output_cost': 4.2e-06, 'total_cost': 5.25e-06})

Testing the Ollama provider:

In [5]:
ollama_provider = OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434")
question = "What is the capital of France?"
asyncio.run(ollama_provider.get_response(question))

('The capital of France is Paris!',
 {'input_cost': 0, 'output_cost': 0, 'total_cost': 0})

Testing the LiteLLM provider:

In [6]:
litellm_provider = LiteLLMProvider(model="gemini/gemini-1.5-flash")

# Other models to try:
# litellm_provider = LiteLLMProvider(model="gpt-4o-mini")
# litellm_provider = LiteLLMProvider(model="claude-3-5-sonnet-20240620")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
# litellm_provider = LiteLLMProvider(model="ollama/llama3.2-vision:latest")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")

question = "What is the capital of France?"
asyncio.run(litellm_provider.get_response(question))

('The capital of France is Paris.\n', 3e-06)

Testing the ScoringReviewer agent:

In [8]:
# Load a dummy dataset

data = pd.read_csv("data.csv")
data

,Title,Abstract,Authors,Year
0,Fusing an agent-based model of mosquito popula...,The mosquito Aedes aegypti is the vector of a ...,"Cavany, S.M.",2022
1,PDRL: Multi-Agent based Reinforcement Learning...,Reinforcement learning has been increasingly a...,"Shaik, T.",2023
2,Learning-accelerated Discovery of Immune-Tumou...,We present an integrated framework for enablin...,"Ozik, J.",2019
3,Investigating spatiotemporal dynamics and sync...,"In this paper we present AceMod, an agent-base...","Cliff, O.M.",2018
4,Modeling the Spread of COVID-19 in University ...,Mathematical and simulation models are often u...,"Herrmann, J.W.",2024
5,Multi-Agent Reinforcement Learning with Action...,Unmanned Aerial Vehicles (UAVs) are increasing...,"Rizvi, D.",2023
6,A new (N) stochastic quenched disorder model f...,Human beings live in a networked world in whic...,"Ferreira, A.A.",2019
7,Frustration induced phases in migrating cell c...,Collective motion of cells is common in many p...,"Copenhagen, K.",2017
8,Universal masking is urgent in the COVID-19 pa...,We present two models for the COVID-19 pandemi...,"Kai, D.",2020
9,Calculus of consent via MARL: Legitimating the...,"Public policies that supply public goods, espe...","Hu, Y.",2021


In [9]:
agent = ScoringReviewer(
    provider=LiteLLMProvider(model="claude-3-5-sonnet-20241022"),
    name="Pouria",
    max_concurrent_requests=1, 
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Search for articles whose titles suggest they are about AI or machine learning agents.",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the article title meets the criteria, and 2 if the article paper does not meet the criteria.',
)


# Dummy input
text_list = data.Title.str.lower().tolist()
print("====== Inputs ======\n\n", '\n'.join(text_list[:3]))

# Dummy review
results, total_cost = asyncio.run(agent.review_items(text_list[:3]))
print("\n====== Outputs ======")
for result in results:
    print(result)

# Dummy costs
print("\n====== Costs ======\n")
for i, item in enumerate(agent.memory):
    print(f"Cost for item {i}: {item['cost']}")

print(f"\nTotal cost: {total_cost}")

====== Inputs ======

 fusing an agent-based model of mosquito population dynamics with a statistical reconstruction of spatio-temporal abundance patterns
pdrl: multi-agent based reinforcement learning for predictive monitoring
learning-accelerated discovery of immune-tumour interactions


Reviewing 3 items - 2024-12-26 23:31:59: 100%|██████████| 3/3 [00:09<00:00,  3.04s/it]


====== Outputs ======
{'reasoning': "While this title mentions 'agent-based model', it refers to ecological modeling of mosquito populations rather than AI/ML agents.", 'score': 2, 'certainty': 95}
{'reasoning': "The title explicitly mentions 'multi-agent based reinforcement learning' which directly indicates it deals with AI/machine learning agents", 'score': 1, 'certainty': 100}
{'reasoning': "The term 'learning-accelerated' in the title directly implies the use of machine learning techniques for discovery in immune-tumor interactions.", 'score': 1, 'certainty': 90}

====== Costs ======

Cost for item 0: 0.003318
Cost for item 1: 0.0032760000000000003
Cost for item 2: 0.003297

Total cost: 0.003297
